# Extracción de Datos desde Delta Lake

Este notebook extrae y analiza todos los datos almacenados en las tablas Delta Lake de Polymarket.

## Tablas disponibles:
- **events**: Eventos de mercados de predicción
- **markets**: Mercados individuales
- **series**: Series de eventos
- **tags**: Etiquetas/categorías

## 1. Instalación de Paquetes Necesarios

Primero instalamos las bibliotecas necesarias para trabajar con Delta Lake.

In [1]:
# Instalar paquetes necesarios (ejecutar solo la primera vez)
# %pip install deltalake pandas pyarrow openpyxl matplotlib seaborn

## 2. Importar Bibliotecas

In [1]:
import pandas as pd
from deltalake import DeltaTable
import os
from pathlib import Path
import json
from datetime import datetime

# Configuración para mostrar más columnas en pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

print("✓ Bibliotecas importadas correctamente")

✓ Bibliotecas importadas correctamente


## 3. Configurar Rutas de Delta Lake

Definimos las rutas a las tablas Delta Lake.

In [2]:
# Ruta base de Delta Lake
BASE_PATH = Path("delta_lake")

# Rutas a cada tabla
PATHS = {
    "events": BASE_PATH / "events",
    "markets": BASE_PATH / "markets",
    "series": BASE_PATH / "series",
    "tags": BASE_PATH / "tags"
}

# Verificar que las rutas existan
print("Verificando rutas de Delta Lake:")
print("-" * 50)
for name, path in PATHS.items():
    exists = path.exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path} - {'Encontrada' if exists else 'No encontrada'}")
print("-" * 50)

Verificando rutas de Delta Lake:
--------------------------------------------------
✓ events: delta_lake\events - Encontrada
✓ markets: delta_lake\markets - Encontrada
✓ series: delta_lake\series - Encontrada
✓ tags: delta_lake\tags - Encontrada
--------------------------------------------------


## 4. RE-EXTRAER TODOS LOS DATOS DE LA API (SIN LÍMITES)

**IMPORTANTE**: Esta sección extrae **TODOS** los datos disponibles de la API de Polymarket sin límites.
- Utiliza paginación automática para obtener datos completos
- Sobrescribe las tablas Delta Lake existentes
- Puede tardar varios minutos dependiendo de la cantidad de datos

**Ejecuta estas celdas SOLO si quieres actualizar los datos con una extracción completa.**

In [ ]:
# Importar los extractores
import sys
sys.path.append('.')

from extract_tags import TagsExtractor
from extract_events import EventsExtractor
from extract_series import SeriesExtractor
from extract_markets import MarketsExtractor
from config import EXTRACTION_CONFIG

print("Configuración actual de extracción:")
print(f"  - Límite por petición: {EXTRACTION_CONFIG['limit']}")
print(f"  - Máximo de registros: {EXTRACTION_CONFIG['max_records']} (0 = SIN LÍMITE)")
print("\n✓ Extractores importados correctamente")

In [ ]:
# PASO 1: Extraer TAGS (todas las etiquetas)
print("=" * 70)
print("EXTRAYENDO TAGS - TODOS LOS REGISTROS")
print("=" * 70)

extractor_tags = TagsExtractor()
all_tags = extractor_tags.extract_all_tags(max_records=0)  # 0 = sin límite

if all_tags:
    print(f"\n✓ Total de tags extraídos: {len(all_tags):,}")
    # Guardar en Delta Lake
    extractor_tags.save_to_delta(all_tags, "tags")
    print(f"✓ Tags guardados en Delta Lake")
else:
    print("\n✗ No se pudieron extraer los tags")
    
print("=" * 70)

In [ ]:
# PASO 2: Extraer EVENTS (todos los eventos)
print("=" * 70)
print("EXTRAYENDO EVENTS - TODOS LOS REGISTROS")
print("=" * 70)

extractor_events = EventsExtractor()
all_events = extractor_events.extract_all_events(max_records=0)  # 0 = sin límite

if all_events:
    print(f"\n✓ Total de events extraídos: {len(all_events):,}")
    # Guardar en Delta Lake
    extractor_events.save_to_delta(all_events, "events")
    print(f"✓ Events guardados en Delta Lake")
else:
    print("\n✗ No se pudieron extraer los events")
    
print("=" * 70)

In [ ]:
# PASO 3: Extraer MARKETS (todos los mercados)
print("=" * 70)
print("EXTRAYENDO MARKETS - TODOS LOS REGISTROS")
print("=" * 70)

extractor_markets = MarketsExtractor()
all_markets = extractor_markets.extract_all_markets(max_records=0)  # 0 = sin límite

if all_markets:
    print(f"\n✓ Total de markets extraídos: {len(all_markets):,}")
    # Guardar en Delta Lake
    extractor_markets.save_to_delta(all_markets, "markets")
    print(f"✓ Markets guardados en Delta Lake")
else:
    print("\n✗ No se pudieron extraer los markets")
    
print("=" * 70)

In [ ]:
# PASO 4: Extraer SERIES (todas las series de eventos)
print("=" * 70)
print("EXTRAYENDO SERIES - TODOS LOS REGISTROS")
print("=" * 70)

extractor_series = SeriesExtractor()
all_series = extractor_series.extract_all_series(max_records=0)  # 0 = sin límite

if all_series:
    print(f"\n✓ Total de series extraídas: {len(all_series):,}")
    # Guardar en Delta Lake
    extractor_series.save_to_delta(all_series, "series")
    print(f"✓ Series guardadas en Delta Lake")
else:
    print("\n✗ No se pudieron extraer las series")
    
print("=" * 70)

In [ ]:
# RESUMEN DE LA EXTRACCIÓN COMPLETA
print("\n")
print("╔" + "═" * 68 + "╗")
print("║" + " RESUMEN DE EXTRACCIÓN COMPLETA ".center(68) + "║")
print("╠" + "═" * 68 + "╣")

resumen = []
if 'all_tags' in locals():
    resumen.append(f"║  Tags:    {len(all_tags):>8,} registros extraídos" + " " * 33 + "║")
if 'all_events' in locals():
    resumen.append(f"║  Events:  {len(all_events):>8,} registros extraídos" + " " * 33 + "║")
if 'all_markets' in locals():
    resumen.append(f"║  Markets: {len(all_markets):>8,} registros extraídos" + " " * 33 + "║")
if 'all_series' in locals():
    resumen.append(f"║  Series:  {len(all_series):>8,} registros extraídos" + " " * 33 + "║")

for linea in resumen:
    print(linea)

total = sum([
    len(all_tags) if 'all_tags' in locals() else 0,
    len(all_events) if 'all_events' in locals() else 0,
    len(all_markets) if 'all_markets' in locals() else 0,
    len(all_series) if 'all_series' in locals() else 0
])

print("╠" + "═" * 68 + "╣")
print(f"║  TOTAL:   {total:>8,} registros extraídos" + " " * 33 + "║")
print("╚" + "═" * 68 + "╝")
print("\n✓ Todos los datos han sido guardados en Delta Lake")
print("✓ Ahora puedes continuar con las siguientes secciones para cargar y analizar los datos")

---

**NOTA**: Si ya ejecutaste las celdas de extracción anteriores, puedes continuar desde aquí para cargar y analizar los datos desde Delta Lake.

---

## 4.5. 🔧 SOLUCIÓN: Reconstruir Tablas Delta Lake Corruptas

**Si obtienes el error "Cannot read metadata from log segment"**, esto indica que los archivos de log de Delta Lake están incompletos o corruptos. 

**Causa del problema:**
- Delta Lake requiere todos los archivos de log desde la versión 0 hasta la versión actual
- Los archivos de log actuales solo contienen las versiones más recientes

**Solución:** Ejecuta las siguientes celdas para:
1. Eliminar completamente las tablas Delta Lake existentes
2. Recrear las tablas desde cero con la extracción completa de datos

In [ ]:
# PASO 1: Eliminar directorios Delta Lake corruptos
import shutil
import os

delta_path = Path("delta_lake")

print("🗑️  ELIMINANDO TABLAS DELTA LAKE CORRUPTAS")
print("=" * 70)

for table_name in ["events", "markets", "series", "tags"]:
    table_dir = delta_path / table_name
    if table_dir.exists():
        try:
            shutil.rmtree(table_dir)
            print(f"✓ Eliminado: {table_dir}")
        except Exception as e:
            print(f"✗ Error al eliminar {table_dir}: {e}")
    else:
        print(f"⊘ No existe: {table_dir}")

print("=" * 70)
print("✓ Directorios eliminados correctamente")
print("\n⚠️ IMPORTANTE: Ahora ejecuta las celdas de la sección 4 (RE-EXTRAER TODOS LOS DATOS)")
print("              para volver a crear las tablas desde cero.")

In [ ]:
# PASO 2: OPCIÓN RÁPIDA - Eliminar y reconstruir TODAS las tablas automáticamente
# ⚠️ ADVERTENCIA: Esto eliminará las tablas existentes y re-extraerá TODOS los datos de la API
# Esta celda hace lo mismo que ejecutar PASO 1 + todas las celdas de la sección 4

import sys
sys.path.append('.')

from extract_tags import TagsExtractor
from extract_events import EventsExtractor
from extract_series import SeriesExtractor
from extract_markets import MarketsExtractor

print("╔" + "═" * 68 + "╗")
print("║" + " RECONSTRUCCIÓN COMPLETA DE DELTA LAKE ".center(68) + "║")
print("╚" + "═" * 68 + "╝")
print()

# 1. Eliminar tablas corruptas
print("🗑️  Paso 1/5: Eliminando tablas corruptas...")
import shutil
delta_path = Path("delta_lake")
for table_name in ["events", "markets", "series", "tags"]:
    table_dir = delta_path / table_name
    if table_dir.exists():
        shutil.rmtree(table_dir)
        print(f"  ✓ {table_name} eliminado")
print()

# 2. Extraer TAGS
print("📥 Paso 2/5: Extrayendo TAGS...")
extractor_tags = TagsExtractor()
all_tags = extractor_tags.extract_all_tags(max_records=0)
if all_tags:
    extractor_tags.save_to_delta(all_tags, "tags")
    print(f"  ✓ {len(all_tags):,} tags guardados")
print()

# 3. Extraer EVENTS
print("📥 Paso 3/5: Extrayendo EVENTS...")
extractor_events = EventsExtractor()
all_events = extractor_events.extract_all_events(max_records=0)
if all_events:
    extractor_events.save_to_delta(all_events, "events")
    print(f"  ✓ {len(all_events):,} events guardados")
print()

# 4. Extraer MARKETS
print("📥 Paso 4/5: Extrayendo MARKETS...")
extractor_markets = MarketsExtractor()
all_markets = extractor_markets.extract_all_markets(max_records=0)
if all_markets:
    extractor_markets.save_to_delta(all_markets, "markets")
    print(f"  ✓ {len(all_markets):,} markets guardados")
print()

# 5. Extraer SERIES
print("📥 Paso 5/5: Extrayendo SERIES...")
extractor_series = SeriesExtractor()
all_series = extractor_series.extract_all_series(max_records=0)
if all_series:
    extractor_series.save_to_delta(all_series, "series")
    print(f"  ✓ {len(all_series):,} series guardadas")
print()

# Resumen
print("╔" + "═" * 68 + "╗")
print("║" + " ✅ RECONSTRUCCIÓN COMPLETADA ".center(68) + "║")
print("╠" + "═" * 68 + "╣")
total = (len(all_tags) if all_tags else 0) + \
        (len(all_events) if all_events else 0) + \
        (len(all_markets) if all_markets else 0) + \
        (len(all_series) if all_series else 0)
print(f"║  Total de registros: {total:,}".ljust(69) + "║")
print("╚" + "═" * 68 + "╝")
print("\n✅ Las tablas Delta Lake han sido reconstruidas correctamente")
print("✅ Ahora puedes continuar con la celda 19 para cargar los datos")

## 5. Función para Cargar Tablas Delta Lake

Creamos una función reutilizable para cargar cualquier tabla Delta Lake.

In [13]:
def load_delta_table(table_path, table_name="tabla"):
    """
    Carga una tabla Delta Lake y la convierte a pandas DataFrame.
    Si Delta Lake falla, intenta leer los archivos Parquet directamente.
    
    Args:
        table_path: Ruta a la tabla Delta Lake
        table_name: Nombre de la tabla para mensajes
        
    Returns:
        pandas DataFrame con los datos
    """
    try:
        print(f"Cargando tabla '{table_name}' desde: {table_path}")
        
        # Cargar Delta Table
        dt = DeltaTable(str(table_path))
        
        # Convertir a pandas DataFrame
        df = dt.to_pandas()
        
        # Información básica
        print(f"  ✓ Registros cargados: {len(df):,}")
        print(f"  ✓ Columnas: {len(df.columns)}")
        print(f"  ✓ Versión de la tabla: {dt.version()}")
        
        return df
        
    except Exception as e:
        # Si Delta Lake falla, intentar leer Parquet directamente
        if "Cannot read metadata from log segment" in str(e) or "log segment" in str(e).lower():
            print(f"  ⚠ Error con Delta Lake: {str(e)}")
            print(f"  → Intentando leer archivos Parquet directamente...")
            
            try:
                # Buscar todos los archivos .parquet en el directorio
                parquet_files = list(table_path.glob("*.parquet"))
                
                if not parquet_files:
                    print(f"  ✗ No se encontraron archivos Parquet en {table_path}")
                    return None
                
                # Leer todos los archivos Parquet y combinarlos
                dfs = []
                for pf in parquet_files:
                    df_part = pd.read_parquet(pf)
                    dfs.append(df_part)
                
                # Concatenar todos los DataFrames
                df = pd.concat(dfs, ignore_index=True)
                
                # Información básica
                print(f"  ✓ Registros cargados (desde Parquet): {len(df):,}")
                print(f"  ✓ Columnas: {len(df.columns)}")
                
                return df
                
            except Exception as e2:
                print(f"  ✗ Error al cargar Parquet: {str(e2)}")
                return None
        else:
            print(f"  ✗ Error al cargar '{table_name}': {str(e)}")
            return None


## 6. Cargar Todas las Tablas

Cargamos todas las tablas Delta Lake disponibles.

### ⚠️ Solución de Problemas

**Si obtienes el error:** `Generic DeltaTable error: Cannot read metadata from log segment`

Esto indica que las tablas Delta Lake están corruptas. **Solución:**

1. Ve a la sección **4.5. SOLUCIÓN: Reconstruir Tablas Delta Lake Corruptas**
2. Ejecuta la celda "OPCIÓN RÁPIDA" para eliminar y reconstruir todas las tablas
3. Regresa aquí y ejecuta la celda 19 nuevamente

### ✅ Solución Alternativa: Leer Parquet Directamente

Si no quieres re-extraer los datos de la API, puedes leer los archivos Parquet directamente sin usar Delta Lake. Esta opción es más rápida y evita problemas con logs corruptos.

In [14]:
def load_parquet_table(table_path, table_name="tabla"):
    """
    Carga archivos Parquet directamente sin usar Delta Lake.
    
    Args:
        table_path: Ruta a la carpeta de la tabla
        table_name: Nombre de la tabla para mensajes
        
    Returns:
        pandas DataFrame con los datos
    """
    try:
        print(f"Cargando tabla '{table_name}' desde archivos Parquet: {table_path}")
        
        # Buscar todos los archivos .parquet en el directorio
        parquet_files = list(table_path.glob("*.parquet"))
        
        if not parquet_files:
            print(f"  ✗ No se encontraron archivos Parquet en {table_path}")
            return None
        
        print(f"  → Encontrados {len(parquet_files)} archivo(s) Parquet")
        
        # Leer todos los archivos Parquet y combinarlos
        dfs = []
        for pf in parquet_files:
            df_part = pd.read_parquet(pf)
            dfs.append(df_part)
            print(f"    ✓ {pf.name}: {len(df_part):,} registros")
        
        # Concatenar todos los DataFrames
        df = pd.concat(dfs, ignore_index=True)
        
        # Información básica
        print(f"  ✓ Total de registros: {len(df):,}")
        print(f"  ✓ Columnas: {len(df.columns)}")
        
        return df
        
    except Exception as e:
        print(f"  ✗ Error al cargar '{table_name}': {str(e)}")
        import traceback
        traceback.print_exc()
        return None

print("✓ Función de lectura directa de Parquet definida")

✓ Función de lectura directa de Parquet definida


In [6]:
# EJECUTA ESTA CELDA EN LUGAR DE LA CELDA 19 (si tienes problemas con Delta Lake)

# Diccionario para almacenar todos los DataFrames
dataframes = {}

print("=" * 60)
print("CARGANDO TABLAS DESDE ARCHIVOS PARQUET (SIN DELTA LAKE)")
print("=" * 60)
print()

# Cargar cada tabla usando lectura directa de Parquet
for table_name, table_path in PATHS.items():
    if table_path.exists():
        df = load_parquet_table(table_path, table_name)
        if df is not None:
            dataframes[table_name] = df
        print()
    else:
        print(f"⚠ Tabla '{table_name}' no encontrada en: {table_path}")
        print()

print("=" * 60)
print(f"RESUMEN: {len(dataframes)} de {len(PATHS)} tablas cargadas correctamente")
print("=" * 60)

# Mostrar resumen de cada tabla
if dataframes:
    print("\n📊 Detalle de tablas cargadas:\n")
    for table_name, df in dataframes.items():
        print(f"  • {table_name.upper()}: {len(df):,} registros, {len(df.columns)} columnas")
    print("\n✅ Datos cargados exitosamente usando archivos Parquet directamente")
    print("✅ Puedes continuar con las siguientes celdas para analizar los datos")
else:
    print("\n⚠️ No se pudieron cargar las tablas. Verifica que los archivos Parquet existan.")

CARGANDO TABLAS DESDE ARCHIVOS PARQUET (SIN DELTA LAKE)

Cargando tabla 'events' desde archivos Parquet: delta_lake\events
  → Encontrados 3 archivo(s) Parquet
    ✓ part-00001-9e196ef7-c450-4225-b8b7-8cb78bc41759-c000.snappy.parquet: 56,320 registros
    ✓ part-00002-9e196ef7-c450-4225-b8b7-8cb78bc41759-c000.snappy.parquet: 87,040 registros
    ✓ part-00003-9e196ef7-c450-4225-b8b7-8cb78bc41759-c000.snappy.parquet: 51,854 registros
  ✓ Total de registros: 195,214
  ✓ Columnas: 92

Cargando tabla 'markets' desde archivos Parquet: delta_lake\markets
  → Encontrados 3 archivo(s) Parquet
    ✓ part-00001-c0b1ed17-c92c-4187-a711-5a9d55277591-c000.snappy.parquet: 140,288 registros
    ✓ part-00002-c0b1ed17-c92c-4187-a711-5a9d55277591-c000.snappy.parquet: 162,816 registros
    ✓ part-00003-c0b1ed17-c92c-4187-a711-5a9d55277591-c000.snappy.parquet: 133,624 registros
  ✓ Total de registros: 436,728
  ✓ Columnas: 140

Cargando tabla 'series' desde archivos Parquet: delta_lake\series
  → Encontrad

In [15]:
# Diccionario para almacenar todos los DataFrames
dataframes = {}

print("=" * 60)
print("CARGANDO TODAS LAS TABLAS DELTA LAKE")
print("=" * 60)
print()

# Cargar cada tabla
for table_name, table_path in PATHS.items():
    if table_path.exists():
        df = load_delta_table(table_path, table_name)
        if df is not None:
            dataframes[table_name] = df
        print()
    else:
        print(f"⚠ Tabla '{table_name}' no encontrada en: {table_path}")
        print()

print("=" * 60)
print(f"RESUMEN: {len(dataframes)} de {len(PATHS)} tablas cargadas correctamente")
print("=" * 60)

CARGANDO TODAS LAS TABLAS DELTA LAKE

Cargando tabla 'events' desde: delta_lake\events
  ⚠ Error con Delta Lake: Generic DeltaTable error: Cannot read metadata from log segment
  → Intentando leer archivos Parquet directamente...
  ✓ Registros cargados (desde Parquet): 195,214
  ✓ Columnas: 92

Cargando tabla 'markets' desde: delta_lake\markets
  ⚠ Error con Delta Lake: Generic DeltaTable error: Cannot read metadata from log segment
  → Intentando leer archivos Parquet directamente...
  ✓ Registros cargados (desde Parquet): 436,728
  ✓ Columnas: 140

Cargando tabla 'series' desde: delta_lake\series
  ✓ Registros cargados: 1,073
  ✓ Columnas: 35
  ✓ Versión de la tabla: 0

Cargando tabla 'tags' desde: delta_lake\tags
  ✓ Registros cargados: 5,027
  ✓ Columnas: 14
  ✓ Versión de la tabla: 0

RESUMEN: 4 de 4 tablas cargadas correctamente


## 7. Información Detallada de las Tablas

Mostramos información detallada sobre cada tabla cargada.

In [16]:
# Crear DataFrame con resumen de todas las tablas
summary_data = []

for table_name, df in dataframes.items():
    summary_data.append({
        'Tabla': table_name.upper(),
        'Registros': f"{len(df):,}",
        'Columnas': len(df.columns),
        'Tamaño (MB)': f"{df.memory_usage(deep=True).sum() / 1024**2:.2f}",
        'Columnas_Lista': ', '.join(df.columns[:5].tolist()) + ('...' if len(df.columns) > 5 else '')
    })

summary_df = pd.DataFrame(summary_data)
print("\n📊 RESUMEN DE TABLAS CARGADAS\n")
print(summary_df.to_string(index=False))
print()


📊 RESUMEN DE TABLAS CARGADAS

  Tabla Registros  Columnas Tamaño (MB)                                       Columnas_Lista
 EVENTS   195,214        92      455.17              id, ticker, slug, title, description...
MARKETS   436,728       140     1303.48 id, question, conditionId, slug, twitterCardImage...
 SERIES     1,073        35        0.64               id, ticker, slug, title, seriesType...
   TAGS     5,027        14        1.31           id, label, slug, publishedAt, createdAt...



### 7.1 Esquema de la Tabla EVENTS

In [17]:
if 'events' in dataframes:
    df_events = dataframes['events']
    print("📋 TABLA: EVENTS")
    print("-" * 80)
    print(f"Registros: {len(df_events):,}")
    print(f"Columnas: {len(df_events.columns)}")
    print("\nTipos de datos:")
    print(df_events.dtypes.to_string())
    print("\nPrimeras columnas:")
    print(df_events.columns.tolist()[:10])

📋 TABLA: EVENTS
--------------------------------------------------------------------------------
Registros: 195,214
Columnas: 92

Tipos de datos:
id                                  str
ticker                              str
slug                                str
title                               str
description                         str
resolutionSource                    str
startDate                           str
creationDate                        str
endDate                             str
image                               str
icon                                str
active                             bool
closed                             bool
archived                         object
new                              object
featured                         object
restricted                       object
liquidity                       float64
volume                          float64
openInterest                    float64
sortBy                              str
category      

### 7.2 Esquema de la Tabla MARKETS

In [18]:
if 'markets' in dataframes:
    df_markets = dataframes['markets']
    print("📋 TABLA: MARKETS")
    print("-" * 80)
    print(f"Registros: {len(df_markets):,}")
    print(f"Columnas: {len(df_markets.columns)}")
    print("\nTipos de datos:")
    print(df_markets.dtypes.to_string())
    print("\nPrimeras columnas:")
    print(df_markets.columns.tolist()[:10])

📋 TABLA: MARKETS
--------------------------------------------------------------------------------
Registros: 436,728
Columnas: 140

Tipos de datos:
id                                         str
question                                   str
conditionId                                str
slug                                       str
twitterCardImage                           str
endDate                                    str
category                                   str
liquidity                                  str
image                                      str
icon                                       str
description                                str
outcomes                                   str
outcomePrices                              str
volume                                     str
active                                    bool
marketType                                 str
closed                                    bool
marketMakerAddress                         str
update

### 7.3 Esquema de la Tabla SERIES

In [19]:
if 'series' in dataframes:
    df_series = dataframes['series']
    print("📋 TABLA: SERIES")
    print("-" * 80)
    print(f"Registros: {len(df_series):,}")
    print(f"Columnas: {len(df_series.columns)}")
    print("\nTipos de datos:")
    print(df_series.dtypes.to_string())
    print("\nColumnas:")
    print(df_series.columns.tolist())

📋 TABLA: SERIES
--------------------------------------------------------------------------------
Registros: 1,073
Columnas: 35

Tipos de datos:
id                                  str
ticker                              str
slug                                str
title                               str
seriesType                          str
recurrence                          str
image                               str
icon                                str
active                             bool
closed                             bool
archived                           bool
createdAt                           str
updatedAt                           str
volume24hr                      float64
volume                          float64
liquidity                       float64
events                           object
commentCount                      int64
requiresTranslation                bool
featured                         object
restricted                       object
new             

### 7.4 Esquema de la Tabla TAGS

In [20]:
if 'tags' in dataframes:
    df_tags = dataframes['tags']
    print("📋 TABLA: TAGS")
    print("-" * 80)
    print(f"Registros: {len(df_tags):,}")
    print(f"Columnas: {len(df_tags.columns)}")
    print("\nTipos de datos:")
    print(df_tags.dtypes.to_string())
    print("\nColumnas:")
    print(df_tags.columns.tolist())

📋 TABLA: TAGS
--------------------------------------------------------------------------------
Registros: 5,027
Columnas: 14

Tipos de datos:
id                                  str
label                               str
slug                                str
publishedAt                         str
createdAt                           str
updatedAt                           str
requiresTranslation                bool
forceShow                        object
isCarousel                       object
updatedBy                       float64
forceHide                        object
createdBy                       float64
_extraction_timestamp    datetime64[us]
_extraction_date                 object

Columnas:
['id', 'label', 'slug', 'publishedAt', 'createdAt', 'updatedAt', 'requiresTranslation', 'forceShow', 'isCarousel', 'updatedBy', 'forceHide', 'createdBy', '_extraction_timestamp', '_extraction_date']


## 8. Vista Previa de los Datos

Mostramos las primeras filas de cada tabla para inspeccionar los datos.

### 8.1 Vista Previa - EVENTS

In [21]:
if 'events' in dataframes:
    print("🔍 EVENTOS - Primeros 3 registros\n")
    # Seleccionar columnas principales para visualización
    cols_to_show = ['id', 'title', 'slug', 'category', 'active', 'closed', 'liquidity', 'volume']
    available_cols = [col for col in cols_to_show if col in df_events.columns]
    display(df_events[available_cols].head(3))

🔍 EVENTOS - Primeros 3 registros



,id,title,slug,category,active,closed,liquidity,volume
0,2890,NBA: Will the Mavericks beat the Grizzlies by more than 5.5 points in their December 4 matchup?,nba-will-the-mavericks-beat-the-grizzlies-by-more-than-5pt5-points-in-their-december-4-matchup,Sports,True,True,0.0,1335.05
1,2891,NFL: Will the Falcons beat the Panthers by more than 3.5 points in their October 31st matchup?,nfl-will-the-falcons-beat-the-panthers-by-more-than-3pt5-points-in-their-october-31st-matchup,Sports,True,True,0.0,5332.42
2,2892,(In-Game Trading) Will the 49ers beat the Packers by more than 3.5 points in their week three ma...,in-game-trading-will-the-49ers-beat-the-packers-by-more-than-3pt5-points-in-their-week-three-mat...,Sports,True,True,0.0,139186.36


### 8.2 Vista Previa - MARKETS

In [12]:
if 'markets' in dataframes:
    print("🔍 MERCADOS - Primeros 3 registros\n")
    # Seleccionar columnas principales para visualización
    cols_to_show = ['id', 'question', 'slug', 'category', 'active', 'closed', 'liquidity', 'volume']
    available_cols = [col for col in cols_to_show if col in df_markets.columns]
    display(df_markets[available_cols].head(3))

🔍 MERCADOS - Primeros 3 registros



,id,question,slug,category,active,closed,liquidity,volume
0,12,Will Joe Biden get Coronavirus before the election?,will-joe-biden-get-coronavirus-before-the-election,US-current-affairs,True,True,0,32257.445115
1,17,"Will Airbnb begin publicly trading before Jan 1, 2021?",will-airbnb-begin-publicly-trading-before-jan-1-2021,Tech,True,True,0,89665.252158
2,18,"Will a new Supreme Court Justice be confirmed before Nov 3rd, 2020?",will-a-new-supreme-court-justice-be-confirmed-before-nov-3rd-2020,US-current-affairs,True,True,0,43279.456005


### 8.3 Vista Previa - SERIES

In [13]:
if 'series' in dataframes:
    print("🔍 SERIES - Primeros 3 registros\n")
    # Seleccionar columnas principales para visualización
    cols_to_show = ['id', 'title', 'slug', 'active', 'liquidity', 'volume']
    available_cols = [col for col in cols_to_show if col in df_series.columns]
    display(df_series[available_cols].head(3))

🔍 SERIES - Primeros 3 registros



,id,title,slug,active,liquidity,volume
0,10781,"Trump, Putin, and Zelensky seen",trump-putin-and-zelensky-seen,True,3857.8048,3629.428405
1,10988,USDJPY Up or Down Daily,usdjpy-up-or-down-daily,True,7.4099,452.020000
2,11111,Bonbast,bonbast,True,4027.5478,69175.988872


### 8.4 Vista Previa - TAGS

In [14]:
if 'tags' in dataframes:
    print("🔍 TAGS - Primeros 5 registros\n")
    display(df_tags.head(5))

🔍 TAGS - Primeros 5 registros



,id,label,slug,publishedAt,createdAt,updatedAt,requiresTranslation,forceShow,isCarousel,updatedBy,forceHide,_extraction_timestamp,_extraction_date
0,671,jto,jto,2023-12-07 19:16:45.979+00,2023-12-07T19:16:45.993Z,2026-02-06T20:01:39.282804Z,False,None,None,NaN,None,2026-02-10 17:51:34.809949,2026-02-10
1,101592,Tom Aspinal,tom-aspinal,NaN,2024-12-31T19:48:27.938591Z,2026-02-06T20:01:39.296759Z,False,None,None,NaN,None,2026-02-10 17:51:34.809949,2026-02-10
2,101115,Preston,preston,NaN,2024-10-28T20:41:17.828152Z,2026-02-06T20:01:39.322562Z,False,None,None,NaN,None,2026-02-10 17:51:34.809949,2026-02-10
3,746,detroit pistons,detroit-pistons,2023-12-18 18:24:38.687+00,2023-12-18T18:24:38.708Z,2026-02-06T20:01:39.323344Z,False,None,None,NaN,None,2026-02-10 17:51:34.809949,2026-02-10
4,1493,spider-man,spider-man,2024-02-27 19:31:50.152+00,2024-02-27T19:31:50.171Z,2026-02-06T20:01:39.389634Z,False,None,None,NaN,None,2026-02-10 17:51:34.809949,2026-02-10


## 9. Análisis Exploratorio Básico

Realizamos un análisis exploratorio de los datos principales.

### 9.1 Análisis de Events

In [22]:
if 'events' in dataframes:
    print("📊 ANÁLISIS DE EVENTOS\n")
    print("-" * 80)
    
    # Eventos activos vs cerrados
    if 'active' in df_events.columns:
        print("\n🔹 Estado de Eventos:")
        print(df_events['active'].value_counts())
    
    if 'closed' in df_events.columns:
        print("\n🔹 Eventos Cerrados:")
        print(df_events['closed'].value_counts())
    
    # Categorías
    if 'category' in df_events.columns:
        print("\n🔹 Top 10 Categorías:")
        print(df_events['category'].value_counts().head(10))
    
    # Estadísticas de volumen y liquidez
    if 'volume' in df_events.columns:
        print("\n🔹 Estadísticas de Volumen:")
        print(df_events['volume'].describe())
    
    if 'liquidity' in df_events.columns:
        print("\n🔹 Estadísticas de Liquidez:")
        print(df_events['liquidity'].describe())

📊 ANÁLISIS DE EVENTOS

--------------------------------------------------------------------------------

🔹 Estado de Eventos:
active
True    195214
Name: count, dtype: int64

🔹 Eventos Cerrados:
closed
True     188885
False      6329
Name: count, dtype: int64

🔹 Top 10 Categorías:
category
Sports                1443
Crypto                 270
US-current-affairs     245
Pop-Culture            182
NBA Playoffs           124
Coronavirus            109
Business               101
NFTs                    75
Chess                   66
Art                     61
Name: count, dtype: int64

🔹 Estadísticas de Volumen:
count    1.444720e+05
mean     3.583848e+05
std      1.295748e+07
min      0.000000e+00
25%      4.003614e+03
50%      1.916850e+04
75%      7.683921e+04
max      3.686335e+09
Name: volume, dtype: float64

🔹 Estadísticas de Liquidez:
count    1.053020e+05
mean     4.496079e+03
std      2.543903e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+

### 9.2 Análisis de Markets

In [23]:
if 'markets' in dataframes:
    print("📊 ANÁLISIS DE MERCADOS\n")
    print("-" * 80)
    
    # Mercados activos vs cerrados
    if 'active' in df_markets.columns:
        print("\n🔹 Estado de Mercados:")
        print(df_markets['active'].value_counts())
    
    if 'closed' in df_markets.columns:
        print("\n🔹 Mercados Cerrados:")
        print(df_markets['closed'].value_counts())
    
    # Tipos de mercado
    if 'marketType' in df_markets.columns:
        print("\n🔹 Tipos de Mercado:")
        print(df_markets['marketType'].value_counts())
    
    # Categorías
    if 'category' in df_markets.columns:
        print("\n🔹 Top 10 Categorías:")
        print(df_markets['category'].value_counts().head(10))

📊 ANÁLISIS DE MERCADOS

--------------------------------------------------------------------------------

🔹 Estado de Mercados:
active
True    436728
Name: count, dtype: int64

🔹 Mercados Cerrados:
closed
True     409473
False     27255
Name: count, dtype: int64

🔹 Tipos de Mercado:
marketType
normal    10028
scalar       71
Name: count, dtype: int64

🔹 Top 10 Categorías:
category
Sports                2552
Crypto                 374
US-current-affairs     361
Pop-Culture            218
Coronavirus            153
Business               137
NBA Playoffs           126
NFTs                   114
Chess                   77
Art                     61
Name: count, dtype: int64


## 10. Exportar Datos

Funciones para exportar los datos a diferentes formatos.

In [24]:
def export_to_csv(df, table_name, output_dir="data/exported"):
    """Exporta un DataFrame a CSV"""
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    filename = output_path / f"{table_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✓ CSV exportado: {filename}")
    return filename

def export_to_json(df, table_name, output_dir="data/exported"):
    """Exporta un DataFrame a JSON"""
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    filename = output_path / f"{table_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    df.to_json(filename, orient='records', indent=2, force_ascii=False)
    print(f"✓ JSON exportado: {filename}")
    return filename

def export_to_excel(dataframes_dict, output_dir="data/exported"):
    """Exporta múltiples DataFrames a un archivo Excel con hojas separadas"""
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    filename = output_path / f"polymarket_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for table_name, df in dataframes_dict.items():
            # Excel tiene límite de 31 caracteres para nombres de hoja
            sheet_name = table_name[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"  ✓ Hoja '{sheet_name}' añadida")
    
    print(f"✓ Excel exportado: {filename}")
    return filename

print("✓ Funciones de exportación definidas")

✓ Funciones de exportación definidas


### 10.1 Exportar a CSV (Descomentar para ejecutar)

In [25]:
# Descomenta las siguientes líneas para exportar a CSV
print("Exportando tablas a CSV...")
for table_name, df in dataframes.items():
    export_to_csv(df, table_name)

Exportando tablas a CSV...
✓ CSV exportado: data\exported\events_20260216_193533.csv
✓ CSV exportado: data\exported\markets_20260216_193645.csv
✓ CSV exportado: data\exported\series_20260216_193759.csv
✓ CSV exportado: data\exported\tags_20260216_193829.csv


### 10.2 Exportar a JSON (Descomentar para ejecutar)

In [19]:
# Descomenta las siguientes líneas para exportar a JSON
# print("Exportando tablas a JSON...")
# for table_name, df in dataframes.items():
#     export_to_json(df, table_name)

### 10.3 Exportar a Excel (Descomentar para ejecutar)

In [20]:
# Descomenta la siguiente línea para exportar todas las tablas a un único Excel
# export_to_excel(dataframes)

## 11. Acceso Directo a los DataFrames

Los datos están disponibles en el diccionario `dataframes` para análisis adicional:

In [26]:
print("📦 DataFrames disponibles para análisis:\n")
print("Variable principal: dataframes (diccionario)")
print("\nAcceso individual:")
for table_name in dataframes.keys():
    print(f"  • dataframes['{table_name}'] → DataFrame con {len(dataframes[table_name]):,} registros")

print("\nEjemplo de uso:")
print("  df_events = dataframes['events']")
print("  df_markets = dataframes['markets']")
print("  df_series = dataframes['series']")
print("  df_tags = dataframes['tags']")

📦 DataFrames disponibles para análisis:

Variable principal: dataframes (diccionario)

Acceso individual:
  • dataframes['events'] → DataFrame con 195,214 registros
  • dataframes['markets'] → DataFrame con 436,728 registros
  • dataframes['series'] → DataFrame con 1,073 registros
  • dataframes['tags'] → DataFrame con 5,027 registros

Ejemplo de uso:
  df_events = dataframes['events']
  df_markets = dataframes['markets']
  df_series = dataframes['series']
  df_tags = dataframes['tags']


## 12. Ejemplos de Consultas Avanzadas

Algunos ejemplos de cómo realizar consultas sobre los datos.

### Ejemplo 1: Top 10 Eventos por Volumen

In [27]:
if 'events' in dataframes and 'volume' in df_events.columns:
    print("🏆 TOP 10 EVENTOS POR VOLUMEN\n")
    top_events = df_events.nlargest(10, 'volume')[['title', 'volume', 'category', 'active']]
    display(top_events)

🏆 TOP 10 EVENTOS POR VOLUMEN



,title,volume,category,active
194746,Presidential Election Winner 2024,3.686335e+09,NaN,True
5259,NBA Champion,1.712133e+09,NaN,True
4089,Super Bowl Champion 2025,1.152274e+09,NaN,True
5053,Champions League Winner,1.001677e+09,NaN,True
4970,Premier League Winner,8.086656e+08,NaN,True
15326,Big Game Champion 2026,7.040968e+08,NaN,True
36179,Fed decision in January?,6.594995e+08,NaN,True
21898,Democratic Presidential Nominee 2028,6.421200e+08,NaN,True
194765,Popular Vote Winner 2024,6.281635e+08,NaN,True
6367,Who will be inaugurated as President?,5.013466e+08,NaN,True


### Ejemplo 2: Filtrar Mercados Activos

In [23]:
if 'markets' in dataframes and 'active' in df_markets.columns:
    active_markets = df_markets[df_markets['active'] == True]
    print(f"📊 MERCADOS ACTIVOS: {len(active_markets):,} de {len(df_markets):,} total\n")
    
    if len(active_markets) > 0:
        cols_to_show = ['question', 'category', 'volume', 'liquidity']
        available_cols = [col for col in cols_to_show if col in active_markets.columns]
        display(active_markets[available_cols].head(5))

📊 MERCADOS ACTIVOS: 500 de 500 total



,question,category,volume,liquidity
0,Will Joe Biden get Coronavirus before the election?,US-current-affairs,32257.445115,0
1,"Will Airbnb begin publicly trading before Jan 1, 2021?",Tech,89665.252158,0
2,"Will a new Supreme Court Justice be confirmed before Nov 3rd, 2020?",US-current-affairs,43279.456005,0
3,"Will Kim Kardashian and Kanye West divorce before Jan 1, 2021?",Pop-Culture,22067.475119,0.179651
4,"Will Coinbase begin publicly trading before Jan 1, 2021?",Crypto,116803.377183,0.367501


### Ejemplo 3: Distribución por Categoría

In [24]:
if 'events' in dataframes and 'category' in df_events.columns:
    print("📈 DISTRIBUCIÓN DE EVENTOS POR CATEGORÍA\n")
    category_dist = df_events['category'].value_counts().head(15)
    
    # Crear visualización simple
    print(category_dist.to_string())
    
    # Si quieres un gráfico, descomenta las siguientes líneas:
    # import matplotlib.pyplot as plt
    # category_dist.plot(kind='barh', figsize=(10, 6))
    # plt.title('Top 15 Categorías de Eventos')
    # plt.xlabel('Cantidad')
    # plt.tight_layout()
    # plt.show()

📈 DISTRIBUCIÓN DE EVENTOS POR CATEGORÍA

category
Sports                198
Crypto                 54
US-current-affairs     52
Pop-Culture            35
Coronavirus            32
Business               20
NBA Playoffs           20
Chess                  18
Art                    18
NFTs                   13
Ukraine & Russia       11
Science                11
Global Politics         7
Olympics                6
Poker                   2


## 13. Resumen Final

Resumen de los datos extraídos y operaciones disponibles.

In [28]:
print("=" * 80)
print("✅ EXTRACCIÓN DE DATOS COMPLETADA")
print("=" * 80)
print()
print("📊 Resumen:")
print(f"  • Tablas cargadas: {len(dataframes)}")
total_records = sum(len(df) for df in dataframes.values())
print(f"  • Total de registros: {total_records:,}")
print()
print("🔧 Operaciones disponibles:")
print("  • Consultas: Usa pandas para filtrar y analizar los datos")
print("  • Exportación: CSV, JSON, Excel (descomenta las celdas de exportación)")
print("  • Visualización: Matplotlib/Seaborn para gráficos")
print()
print("💾 Los DataFrames están almacenados en la variable 'dataframes'")
print("   Ejemplo: dataframes['events'], dataframes['markets'], etc.")
print("=" * 80)

✅ EXTRACCIÓN DE DATOS COMPLETADA

📊 Resumen:
  • Tablas cargadas: 4
  • Total de registros: 638,042

🔧 Operaciones disponibles:
  • Consultas: Usa pandas para filtrar y analizar los datos
  • Exportación: CSV, JSON, Excel (descomenta las celdas de exportación)
  • Visualización: Matplotlib/Seaborn para gráficos

💾 Los DataFrames están almacenados en la variable 'dataframes'
   Ejemplo: dataframes['events'], dataframes['markets'], etc.
